In [149]:
import os
import glob
import json
import pandas as pd

### Load File JSON dari Folder

In [150]:
# Mencari file JSON Terlebih dahulu
def load_json_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

### Read File JSON

In [151]:
# Membaca isi file dari setiap file JSON 
def load_json_list_to_df(filelist):
    all_records = []
    for file in filelist:
        with open(file, "r") as f:
            data = json.load(f)
            all_records.append(data)
    df = pd.json_normalize(all_records)
    if "ts" in df.columns:
        df = df.sort_values("ts").reset_index(drop=True)
    return df

### 1. Visualize the complete historical table view of each tables in tabular format in stdout (hint: print your table)

In [152]:
accounts = load_json_files("../data/accounts")
cards = load_json_files("../data/cards")
savings_accounts = load_json_files("../data/savings_accounts")

accounts_df = load_json_list_to_df(accounts)
cards_df = load_json_list_to_df(cards)
savings_df = load_json_list_to_df(savings_accounts)

print("Accounts DataFrame:")
print(accounts_df.to_string())
print("\nCards DataFrame:")
print(cards_df.to_string())
print("\nSavings Accounts DataFrame:")
print(savings_df.to_string())

Accounts DataFrame:
           id op             ts data.account_id data.name data.address data.phone_number            data.email set.phone_number set.savings_account_id set.address                set.email set.card_id
0  a1globalid  c  1577863800000              a1   Anthony     New York          12345678  anthony@somebank.com              NaN                    NaN         NaN                      NaN         NaN
1  a1globalid  u  1577865600000             NaN       NaN          NaN               NaN                   NaN         87654321                    NaN         NaN                      NaN         NaN
2  a1globalid  u  1577890800000             NaN       NaN          NaN               NaN                   NaN              NaN                    sa1         NaN                      NaN         NaN
3  a1globalid  u  1577894400000             NaN       NaN          NaN               NaN                   NaN              NaN                    NaN     Jakarta  anthony@anotherb

### 2. Visualize the complete historical table view of the denormalized joined table in stdout by joining these three tables (hint: the join key lies in the resources section, please read carefully)

In [153]:
def merge_cdc_dataframes_with_sequence(accounts_df, cards_df, savings_df):

    # Step 1: Buat copy untuk tidak memodifikasi original
    accounts = accounts_df.copy()
    cards = cards_df.copy()
    savings = savings_df.copy()
    
    # Step 2: Tambahkan sequence number ke setiap DataFrame
    # Sequence ini akan menjaga urutan original dalam setiap source
    accounts['seq'] = range(len(accounts))
    cards['seq'] = range(len(cards))
    savings['seq'] = range(len(savings))
    
    # Step 3: Tambahkan source identifier untuk tracking
    accounts['source'] = 'accounts'
    cards['source'] = 'cards'
    savings['source'] = 'savings'
    
    # Step 4: Gabungkan semua DataFrame
    combined_df = pd.concat(
        [accounts, cards, savings], 
        ignore_index=True
    )
    
    # Step 5: Sort berdasarkan timestamp, kemudian sequence
    # Gunakan stable sort untuk mempertahankan urutan relatif
    final_df = combined_df.sort_values(
        by=['ts', 'seq'],
        ascending=[True, True],
        kind='stable'  # Stable sort algorithm
    ).reset_index(drop=True)
    
    return final_df

result_df = merge_cdc_dataframes_with_sequence(accounts_df, cards_df, savings_df)
print("Complete Historical Table View - Denormalized")
print(result_df.head(50).to_string())


Complete Historical Table View - Denormalized
             id op             ts data.account_id data.name data.address data.phone_number            data.email set.phone_number set.savings_account_id set.address                set.email set.card_id  seq    source data.card_id data.card_number  data.credit_used  data.monthly_limit data.status set.status  set.credit_used data.savings_account_id  data.balance  data.interest_rate_percent  set.balance  set.interest_rate_percent
0    a1globalid  c  1577863800000              a1   Anthony     New York          12345678  anthony@somebank.com              NaN                    NaN         NaN                      NaN         NaN    0  accounts          NaN              NaN               NaN                 NaN         NaN        NaN              NaN                     NaN           NaN                         NaN          NaN                        NaN
1    a1globalid  u  1577865600000             NaN       NaN          NaN               NaN  

### 3.From result from point no 2, discuss how many transactions has been made, when did each of them occur, and how much the value of each transaction? Transaction is defined as activity which change the balance of the savings account or credit used of the card

| No | Timestamp       | Jenis   | Nilai Transaksi |
|----|-----------------|---------|-----------------|
| 1  | 1577955600000   | Savings | 15000.0         |
| 2  | 1578313800000   | Card    | 12000.0         |
| 3  | 1578420000000   | Card    | 19000.0         |
| 4  | 1578648600000   | Savings | 40000.0         |
| 5  | 1578654000000   | Card    | 0.0             |
| 6  | 1578654000000   | Savings | 21000.0         |
| 7  | 1579361400000   | Card    | 37000.0         |
| 8  | 1579505400000   | Savings | 33000.0         |

**Total transaksi: 8**